In this notebook, I'm trying to solve a puzzle game we have sitting around the house I grew up in; it's a puzzle where you have to combine some tetronimo-like shapes into a cube.

It's likely a variant of the Soma cube, a puzzle made by Piet Hein.

I started this code over winter holiday in December 2017, so my only notes are the pieces below, which aren't a match to the typical pieces of a Soma cube.

Here are the pieces:

In [88]:
import numpy as np

def piece(*args):
    return np.array([
        [
            ch == 'x'
            for ch in arg
        ]
        for arg in args
    ])

L = piece(
    'xxx',
    'x  ',
)

pieces = [
    piece(
        'xx ',
        ' xx',
    ),
    piece(
        ' x ',
        'xxx',
    ),
    piece(
        'x ',
        'xx',
    ),
    L,
    L,
    L,
    L,
]

def show_piece(p):
    return '\n'.join(''.join(np.where(row, 'x', '.')) for row in p)

print(show_piece(L))

xxx
x..


We first enumerate all shape orientations.

In [116]:
def orientations(shape):
    rev = slice(None, None, -1)
    return [
        shape[:, :],
        shape[rev, :],
        shape[:, rev],
        shape[rev, rev],
    ]
for o in orientations(L):
    print(show_piece(o))
    print()

xxx
x..

x..
xxx

xxx
..x

..x
xxx



Now, given some existing arrangement and a piece we hope to place, we enumerate candidate locations.

In [118]:
def candidate_locations(space, piece):
    sh = space.shape
    psh = piece.shape
    os = orientations(piece)

    for dim1, dim2, dim3 in [
        (0, 1, 2),
        (0, 2, 1),
        (1, 2, 0),
    ]:
        # for all placements
        for i in range(sh[dim1]-psh[0]+1):
            for j in range(sh[dim2]-psh[1]+1):
                for k in range(sh[dim3]):
                    idx = [slice(None), slice(None), slice(None)]
                    idx[dim1] = slice(i, psh[0]+i)
                    idx[dim2] = slice(j, psh[1]+j)
                    idx[dim3] = k
                    idx = tuple(idx)
                    # for all shape orientations
                    for o in os:
                        if not np.any(space[idx] & o):
                            yield idx, o

assert len(list(candidate_locations(np.zeros((3, 3, 3)).astype(np.bool), L))) == 2*1*3*3*4

assert len(list(candidate_locations(np.array([[
    [0, 1, 0],
    [0, 0, 0],
]]).astype(np.bool), L))) == 2

assert len(list(candidate_locations(np.array([[
    [1, 1, 0],
    [0, 0, 0],
]]).astype(np.bool), L))) == 1

assert len(list(candidate_locations(np.array([[
    [1, 1, 0],
    [1, 0, 0],
]]).astype(np.bool), L))) == 0

Now we write a simple wrapper around the deterministic MDP this gives us.

In [119]:
class Problem(object):
    def __init__(self, initial_space, pieces):
        self.initial_space = initial_space
        self.pieces = pieces
    def initial_state(self):
        return self.initial_space, 0
    def is_terminal(self, state):
        space, used_count = state
        return np.all(space) # all must be filled!
    def actions(self, state):
        space, used_count = state
        piece_idx = used_count
        if piece_idx >= len(self.pieces):
            return []
        piece = self.pieces[piece_idx]
        return [
            (piece_idx, space_idx, orientation)
#            for piece_idx, piece in enumerate(self.pieces)
#            if piece_idx not in used
            for space_idx, orientation in candidate_locations(space, piece)
        ]
    def transition(self, state, action):
        space, used_count = state
        piece_idx, space_idx, orientation = action
        space = np.copy(space)
        space[space_idx] |= orientation
#        return (space, used | {piece_idx})
        return (space, used_count + 1)

p = Problem(np.array([[
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0],
]]), [L, L])
actions = p.actions(p.initial_state())
for a in actions:
    ns = p.transition(p.initial_state(), a)
    print('used', ns[1])
    print(show_piece(ns[0][0]))
    print()

used 1
xxx
xx.
...

used 1
xxx
.xx
...

used 1
...
xx.
xxx

used 1
...
.xx
xxx



And a small search routine to give us an answer

In [120]:
def dfs(problem, state=None):
    state = state or problem.initial_state()
    if problem.is_terminal(state):
        return [state]
    for action in problem.actions(state):
        result = dfs(problem, problem.transition(state, action))
        if result is not None:
            return [state, action] + result

res = dfs(Problem(np.array([[
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0],
]]), [L, L]))
assert np.all(res[-1][0])
assert res[-1][1] == 2

assert dfs(Problem(np.array([[
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]]), [L, L])) is None

Now, we try it on the original problem:

In [121]:
res = dfs(Problem(np.zeros((3, 3, 3)).astype(np.bool), pieces))

In [122]:
states = res[::2]
prev = states[0]
for s in states[1:]:
    space, used = s
    summed = space.astype(np.int)+prev[0].astype(np.int)
    summed = np.char.mod('%d', summed)
    summed[summed=='0'] = '.'
    summed[summed=='1'] = '*'
    summed[summed=='2'] = 'o'
    for i in range(space.shape[0]):
        for j in range(space.shape[1]):
            print(''.join(str(x) for x in summed[i, j]), end='  ')
        print()
    print()
    prev = s

*..  *..  ...  
...  *..  *..  
...  ...  ...  

o..  o*.  ...  
.*.  o*.  o*.  
...  ...  ...  

o**  oo.  ...  
.o*  oo.  oo.  
...  ...  ...  

ooo  oo.  ...  
*oo  oo.  oo.  
***  ...  ...  

ooo  oo.  ...  
ooo  oo*  oo.  
ooo  ***  ...  

ooo  oo.  ...  
ooo  ooo  oo*  
ooo  ooo  ***  

ooo  oo*  ***  
ooo  ooo  ooo  
ooo  ooo  ooo  



It seems the Soma cube has different shapes than those I wrote above; not sure if this was a simplification of mine or the puzzle I was working with. I started sketching out an update to the code to make 3D shapes work, but will leave it here for now.

In [123]:
np.stack([piece(
    'xx',
    ' x',
), piece(
    '  ',
    ' x',
)])

array([[[ True,  True],
        [False,  True]],

       [[False, False],
        [False,  True]]])

In [124]:
def candidate_locations_3D(space, piece):
    sh = space.shape

    # hardcoding this
    assert piece.ndim in (2, 3)
    if piece.ndim == 2:
        piece = np.expand_dims(piece, axis=-1)
    psh = piece.shape
    os = orientations(piece)

    for dim1, dim2, dim3 in [
        (0, 1, 2),
        (0, 2, 1),
        (1, 2, 0),
    ]:
        # for all placements
        for i in range(sh[dim1]-psh[0]+1):
            for j in range(sh[dim2]-psh[1]+1):
                for k in range(sh[dim3]-psh[2]+1):
                    idx = [slice(None), slice(None), slice(None)]
                    idx[dim1] = slice(i, psh[0]+i)
                    idx[dim2] = slice(j, psh[1]+j)
                    idx[dim3] = slice(k, k+1)
                    # HACK do tuple instead
                    idx = tuple(idx)
                    # for all shape orientations
                    for o in os:
                        # We reorder the shape so it can be applied in the transition function later.
                        o = np.moveaxis(o, list(range(3)), (dim1, dim2, dim3))
                        if not np.any(space[idx] & o):
                            yield idx, o